In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [2]:
URL = "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

In [3]:
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travelsName___61bd75")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "busType___44b170")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "boardingTime___6bc587")
            duration_elements = driver.find_elements(By.CLASS_NAME, "duration___da3927")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "droppingTime___a452f5")
            star_rating_elements = driver.find_elements(By.XPATH, "rating___f7ba20")
            price_elements = driver.find_elements(By.CLASS_NAME, "finalFare___c05744")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'totalSeats___5f5f18') or contains(@class, 'seatsWrap___1a5212')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

driver = None

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('chandigarh_bus_details.csv', index=False)

# Close the driver
if driver is not None:
    driver.quit()

Error occurred while accessing page 3: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff6b36dfe75+79173]
	GetHandleVerifier [0x0x7ff6b36dfed0+79264]
	(No symbol) [0x0x7ff6b3499e5a]
	(No symbol) [0x0x7ff6b34f0586]
	(No symbol) [0x0x7ff6b34f083c]
	(No symbol) [0x0x7ff6b3544247]
	(No symbol) [0x0x7ff6b35189af]
	(No symbol) [0x0x7ff6b354100d]
	(No symbol) [0x0x7ff6b3518743]
	(No symbol) [0x0x7ff6b34e14c1]
	(No symbol) [0x0x7ff6b34e2253]
	GetHandleVerifier [0x0x7ff6b39aa2ad+3004797]
	GetHandleVerifier [0x0x7ff6b39a46fd+2981325]
	GetHandleVerifier [0x0x7ff6b39c3350+3107360]
	GetHandleVerifier [0x0x7ff6b36fa9fe+188622]
	GetHandleVerifier [0x0x7ff6b370228f+219487]
	GetHandleVerifier [0x0x7ff6b36e8dc4+115860]
	GetHandleVerifier [0x0x7ff6b36e8f79+116297]
	GetHandleVerifier [0x0x7ff6b36cf528+11256]
	BaseThreadInitThunk [0x0x7ffa2bb7257d+29]
	RtlUserThreadStart [0x0x7ffa2c54af08+40]

Error occurred while accessing page 4: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff6b36dfe75+79173]
	GetHandle

In [4]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,Scania Multi-Axle AC Semi Sleeper (2+2),13:30,5h 45m,19:15,0,₹360,7 Seats
1,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,NueGo,Electric A/C Seater/Sleeper (2+1),13:55,4h 35m,18:30,0,₹399,5 Seats
2,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Vacation Express,Volvo 9600 A/C Semi Sleeper (2+2),14:00,5h 10m,19:10,0,₹428,8 Seats
3,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Khurana Bus Service,Volvo A/C (2+2),21:50,5h 10m,03:00,0,₹300,19 Seats
4,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,NueGo,Electric A/C Seater (2+2),14:30,5h 30m,20:00,0,₹447,20 Seats
...,...,...,...,...,...,...,...,...,...,...
71,Chandigarh to Dehradun,https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,Bharat Benz A/C Semi Sleeper (2+2),23:55,5h 20m,05:15,0,"₹1,065",7 Seats
72,Chandigarh to Dehradun,https://www.redbus.in/bus-tickets/chandigarh-t...,Mahalaxmi Travels,Mercedes Multi-Axle Semi Sleeper (2+2),23:00,5h 30m,04:30,0,₹599,28 Seats
73,Chandigarh to Dehradun,https://www.redbus.in/bus-tickets/chandigarh-t...,SHIV BHOLE TOUR AND TRAVELS,A/C Sleeper (2+1),22:30,6h 43m,05:13,0,"₹4,286",22 Seats\n(6 Single)
74,Chandigarh to Dharamshala (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,Volvo 9600 A/C Semi Sleeper (2+2),23:45,5h 45m,05:30,0,"₹1,069",12 Seats
